Based on https://github.com/CodedK/CUDA-by-Example-source-code-for-the-book-s-examples-/blob/master/chapter05/dot.cu

In [1]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.7 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp39-cp39-linux_x86_64.whl size=661963 sha256=405576dcb34c5d04ce8d09dc7a14a40088c1c614390a2665cf29fb6d5ea3cd9a
  Stored in directory: /root/.cache/pip/wheels/82/09/27/74d8e31ed19c530166e0d263aabe1ea57465e255615bda8fc0
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69866 sha256=36298043d38eef51d55ffe42939480d481c0fd1264f440a614be9496b036a39c
  Stored in directory: /root/.cache/pip/wheels/76/ee/8

In [2]:
%%writefile dot_prod.cu

#include <cuda_runtime.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

const int threadsPerBlock = 1024; //must be a power of 2!

extern "C" __global__ void dot_prod_ker(int dim, float *a, float *b, float *c )
{
  __shared__ float cache[threadsPerBlock];
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int cacheIndex = threadIdx.x;

    float   temp = 0;
    while (tid < dim) {
        temp += a[tid] * b[tid];
        tid += blockDim.x * gridDim.x;
    }
    
    // set the cache values
    cache[cacheIndex] = temp;
    
    // synchronize threads in this block
    __syncthreads();

    // Reduction starts here!
    // We have blockDim.x * gridDim.x num of threads, 
    // each containing a partial sum
    
    // for reductions, threadsPerBlock must be a power of 2
    // because of the following code
    int i = blockDim.x/2; //e.g 256/2
    // Let us recursively sum the threads in the blocks
    // The lower half gets the increade by the values from the upper half
    while (i != 0) {
        if (cacheIndex < i)
            cache[cacheIndex] += cache[cacheIndex + i];
        __syncthreads();
        i /= 2;
    }

    if (cacheIndex == 0)
        c[blockIdx.x] = cache[0];
}

Writing dot_prod.cu


In [3]:
!nvcc -ptx -o dot_prod.ptx dot_prod.cu

In [4]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda import gpuarray
from pycuda.compiler import SourceModule
from time import time

import numpy as np

In [5]:
my_mod = pycuda.driver.module_from_file('./dot_prod.ptx')
dot_prod_ker = my_mod.get_function('dot_prod_ker')


In [6]:
dim=100000000

threadsPerBlock = 1024; #this have to be a power of 2
blocksPerGrid=32

print(f'num of blocks: {blocksPerGrid}')
print(f'num of threads in each block: {threadsPerBlock}')
print(f'total number of threads in the grid: {blocksPerGrid*threadsPerBlock}')
print(f'dim of vectors: {dim}')

num of blocks: 32
num of threads in each block: 1024
total number of threads in the grid: 32768
dim of vectors: 100000000


In [7]:

veca=np.float32(np.random.randn(dim))
#veca=np.float32(2.0*np.ones(dim))
#vecb=np.float32(3.0*np.ones(dim))
vecb=np.float32(np.random.randn(dim))

vecc=np.float32(np.zeros(blocksPerGrid))

veca_gpu=gpuarray.to_gpu(veca)
vecb_gpu=gpuarray.to_gpu(vecb)
vecc_gpu=gpuarray.to_gpu(vecc)

print(len(veca), len(vecb), len(vecc))

100000000 100000000 32


In [8]:
t=time()
result=np.dot(veca,vecb)
tCPU=time()-t
print(f'dot product: {result}')
print(f'Elapsed time on CPU: {tCPU}')


dot product: 2319.245361328125
Elapsed time on CPU: 0.06168532371520996


In [9]:
t=time()
dot_prod_ker(np.int32(dim), veca_gpu,vecb_gpu,vecc_gpu,grid=(blocksPerGrid, 1, 1), block=(threadsPerBlock,1,1))
partial_sums_in_blocks=vecc_gpu.get()
result=np.sum(partial_sums_in_blocks)
tGPU=time()-t
print(f'Elapsed time on GPU: {tGPU}')
print(result)

Elapsed time on GPU: 0.0033371448516845703
2319.069


In [10]:
print(f' GPU was {tCPU/tGPU} faster than CPU')

 GPU was 18.484460955919126 faster than CPU


In [11]:
print(partial_sums_in_blocks)

[-1336.9792   -1993.2014   -2156.702     2223.04       182.18013
  2755.8892    1312.0488    2456.3765    -328.09152   2733.114
 -2319.1826     930.0247    -879.2849    -553.6356    -559.4678
  -333.90768    353.80548   1641.1185     877.84546     15.265015
 -3202.4907    -602.26465    290.2035    -907.8406      31.60608
  -956.01434   3224.287     -129.60205    195.94568  -1124.1642
  1882.0198   -1402.8711  ]
